In [1]:
## Set path
import os
from pathlib import Path

current_directory = Path.cwd()
if current_directory.name == "MT":
    # This means that the notebook is run from the main anomalib directory.
    root_directory = current_directory.parent
elif current_directory.name == "anomalib":
    # This means that the notebook is run from the main anomalib directory.
    root_directory = current_directory
    print("here")

os.chdir(root_directory)
root_directory

PosixPath('/home/wueesmat/MT/anomalib')

In [2]:
# Import the required modules
import numpy as np
from sklearn.preprocessing import minmax_scale
from lightning.pytorch import seed_everything
from anomalib.data.image.mvtec import MVTec_contaminated, MVTecDataset_contaminated
from anomalib.models import Patchcore
from anomalib.engine import Engine
from anomalib import TaskType
from anomalib.data.utils import Split

Could not find wandb. To use this feature, ensure that you have wandb installed.
Could not find openvino. To use this feature, ensure that you have openvino installed.
OpenVINO is not installed. Please install OpenVINO to use OpenVINOInferencer.


In [ ]:
#def train_and_predict_on_training_set_blind(coreset_sampling_ratio, run, category, cont_ratio):
#    seed_everything(run, workers=True)
#    datamodule = MVTec_contaminated(category=category, cont_ratio=cont_ratio, run=run, idx=[])
#    model = Patchcore(coreset_sampling_ratio=coreset_sampling_ratio)
#    engine = Engine(task=TaskType.CLASSIFICATION, image_metrics=["AUROC", "AUPR", "F1Score"], max_epochs=10, devices=1)
#    engine.fit(datamodule=datamodule, model=model)
#
#    # Predict on complete training set
#    seed_everything(run, workers=True)
#    train_dataset = MVTecDataset_contaminated(
#                    task=TaskType.CLASSIFICATION,
#                    split=Split.TRAIN,
#                    category=category,
#                    cont_ratio=cont_ratio,
#                    run=run,
#                    idx = []           
#                )
#    predictions_subset = engine.predict(model=model, dataset=train_dataset)
#    
#    return predictions_subset

In [3]:
def train_and_predict_on_training_set_blind(coreset_sampling_ratio, run, category, cont_ratio):
    seed_everything(run, workers=True)
    datamodule = MVTec_contaminated(category=category, cont_ratio=cont_ratio, run=run, idx=[])
    model = Patchcore(coreset_sampling_ratio=coreset_sampling_ratio)
    engine = Engine(task=TaskType.CLASSIFICATION, image_metrics=["AUROC", "AUPR", "F1Score"], max_epochs=10, devices=1)
    engine.fit(datamodule=datamodule, model=model)

    # Predict on complete training set
    seed_everything(run, workers=True)
    train_dataset = MVTecDataset_contaminated(
                    task=TaskType.CLASSIFICATION,
                    split=Split.TRAIN,
                    category=category,
                    cont_ratio=cont_ratio,
                    run=run,
                    idx = []           
                )
    predictions = engine.predict(model=model, dataset=train_dataset)
    S_blind = np.array([d["pred_scores"][0] for d in predictions])
    S_blind = minmax_scale(S_blind, feature_range=(0,1))
    Y_blind = np.array([item['label'].item() for item in predictions])
    
    return S_blind, Y_blind


In [4]:
def train_and_predict_on_training_set_clean(coreset_sampling_ratio, run, category, cont_ratio):
    seed_everything(run, workers=True)
    datamodule = MVTec_contaminated(category=category, cont_ratio=0, run=run, idx=[])
    model = Patchcore(coreset_sampling_ratio=coreset_sampling_ratio)
    engine = Engine(task=TaskType.CLASSIFICATION, image_metrics=["AUROC", "AUPR", "F1Score"], max_epochs=10, devices=1)
    engine.fit(datamodule=datamodule, model=model)

    # Predict on complete training set
    seed_everything(run, workers=True)
    train_dataset = MVTecDataset_contaminated(
                    task=TaskType.CLASSIFICATION,
                    split=Split.TRAIN,
                    category=category,
                    cont_ratio=cont_ratio,
                    run=run,
                    idx = []           
                )
    predictions = engine.predict(model=model, dataset=train_dataset)
    S_clean = np.array([d["pred_scores"][0] for d in predictions])
    S_clean = minmax_scale(S_clean, feature_range=(0,1))
    Y_clean = np.array([item['label'].item() for item in predictions])

    return S_clean, Y_clean

In [ ]:
def train_and_predict_on_training_set_SRR_light(coreset_sampling_ratio, run, category, cont_ratio):

    k = 5
    gamma = 1 - cont_ratio

    train_dataset = MVTecDataset_contaminated(
                    task=TaskType.CLASSIFICATION,
                    split=Split.TRAIN,
                    category=category,
                    cont_ratio=cont_ratio,
                    run=run,
                    idx = []           
                )

    
    # Create indices for k disjoint datasets
    train_dataset_length = train_dataset.__len__()
    print("train_dataset_length:", train_dataset_length)
    indices = np.arange(0, train_dataset_length)
    np.random.seed(run)
    np.random.shuffle(indices)
    indices_disjoint_datasets = np.array_split(indices, k)
    print("indices_disjoint_datasets: ", indices_disjoint_datasets)

    # Train k models on k disjoint datasets
    classifications_subset_arr = np.empty([train_dataset_length,k], dtype=bool)
    for k_iter in range(k):
        print("k_iter: ", k_iter)
        # Train model on disjoint dataset
        seed_everything(run, workers=True)
        datamodule = MVTec_contaminated(category=category, cont_ratio=cont_ratio, run=run, idx=indices_disjoint_datasets[k_iter])
        model = Patchcore(coreset_sampling_ratio=coreset_sampling_ratio) 
        engine = Engine(task=TaskType.CLASSIFICATION, image_metrics=["AUROC", "AUPR", "F1Score"], max_epochs=10, devices=1)
        engine.fit(datamodule=datamodule, model=model)

        # Predict binary labels for each sample
        predictions_subset = engine.predict(model=model, dataset=train_dataset)
        prediction_scores_subset = np.array([d["pred_scores"][0] for d in predictions_subset])
        print("prediction_scores_subset: ", prediction_scores_subset)
        print("gamma:", gamma)
        threshold = np.percentile(prediction_scores_subset, q=gamma*100)
        print("threshold: ", threshold)
        classifications_subset = prediction_scores_subset>threshold # True: abnormal; False: normal
        print("classifications_subset: ", classifications_subset)
        # Save binary classifications
        classifications_subset_arr[:,k_iter] = classifications_subset

    # Return indices of refined dataset
    keep_bool_arr = np.all(~classifications_subset_arr, axis=1)
    keep_indices = np.where(keep_bool_arr)[0]
    print("keep_indices: ", keep_indices)




    # Extract labels
    Y_SRR = np.array([item['label'].item() for item in predictions_subset[0]])
    Y_hat = np.where(~keep_bool_arr)[0]





    return S_SRR, Y_SRR

In [5]:
def train_and_predict_on_training_set_USDR(coreset_sampling_ratio, run, category, cont_ratio):

    seed_everything(run, workers=True)
    train_dataset = MVTecDataset_contaminated(
                    task=TaskType.CLASSIFICATION,
                    split=Split.TRAIN,
                    category=category,
                    cont_ratio=cont_ratio,
                    run=run,
                    idx = []           
                )

    # Create indices for k disjoint datasets
    train_dataset_length = train_dataset.__len__()

    # Define hyperparameters
    N = train_dataset_length#23
    M = 8#16
    M_train = int(M/2)

    # Create subset boolean matrix
    ones = np.ones(M_train, dtype=bool)
    zeros = np.zeros(M-M_train, dtype=bool)
    period_bool = np.hstack((ones, zeros))
    subset1_bool = np.array([], dtype=bool)
    for i in range(int(np.ceil(N/M))):
        subset1_bool = np.hstack((subset1_bool, period_bool))
    subsets_bool = subset1_bool
    for m in range(1,M):
        subset_new_bool = np.roll(subset1_bool, shift=m)
        subsets_bool = np.vstack((subsets_bool, subset_new_bool))
    subsets_bool = subsets_bool[:,:int(np.floor(N/M)*M + np.mod(N, M))]
    print(subsets_bool.astype(int))

    # Initialize arrays for predictions and subsets
    predictions_subsets_arr = np.empty(M, dtype=object)
    subsets_bool_sorted_arr = np.empty((M,N))

    # Create indices for M partially overlapping subsets
    indices = np.arange(0, train_dataset_length)
    np.random.seed(run)
    np.random.shuffle(indices)

    # Train M models on M partially overlapping subsets
    for m_iter in range(M):

        # Define indices for M partially overlapping subsets
        indices_subset = np.sort(indices[np.where(subsets_bool.astype(int)[m_iter])])
        
        # Train model on subset
        seed_everything(run, workers=True)
        datamodule = MVTec_contaminated(category=category, cont_ratio=cont_ratio, run=run, idx=indices_subset)
        model = Patchcore(coreset_sampling_ratio=coreset_sampling_ratio) 
        engine = Engine(task=TaskType.CLASSIFICATION, image_metrics=["AUROC", "AUPR", "F1Score"], max_epochs=10, devices=1)
        engine.fit(datamodule=datamodule, model=model)

        # Predict complete training set
        predictions_subset = engine.predict(model=model, dataset=train_dataset)

        # Save the predictions
        predictions_subsets_arr[m_iter] = predictions_subset

        # Save the subset
        subsets_bool_sorted = np.zeros(N)
        subsets_bool_sorted[indices_subset] = 1
        subsets_bool_sorted_arr[m_iter,:] = subsets_bool_sorted

    # Extract 'pred_scores' from each list and stack them
    pred_scores_list = []
    for sublist in predictions_subsets_arr:
        stacked_sublist_scores = np.array([d['pred_scores'].item() for d in sublist])
        pred_scores_list.append(stacked_sublist_scores)
    pred_scores_arr = np.vstack(pred_scores_list)

    # Extract labels
    label_arr = np.array([item['label'].item() for item in predictions_subsets_arr[0]])

    # Compute USDR scores
    Y = label_arr 
    Y_hat = pred_scores_arr
    Residuals = abs(Y_hat) # abs(Y - Y_hat)
    Included = subsets_bool_sorted_arr
    NotIncluded = 1-subsets_bool_sorted_arr
    N_j = np.sum(Included, axis=1)
    mu_j = np.empty([M])
    std_j = np.empty([M])
    for j in range(M):
        mu_j[j] = np.mean(Residuals[j,:][Included[j,:]==1])
        std_j[j] = np.std(Residuals[j,:][Included[j,:]==1])
    Rescaled_Residuals = (Residuals - mu_j[:, np.newaxis])/std_j[:, np.newaxis]
    Z_train = 1/M_train*np.sum(Rescaled_Residuals*Included, axis=0)
    Z_infer = 1/(M-M_train)*np.sum(Rescaled_Residuals*NotIncluded, axis=0)
    S_USDR = Z_infer - Z_train
    S_USDR = minmax_scale(S_USDR, feature_range=(0,1))

    # Compute Alternative USDR score
    Z_train_min = np.min(Rescaled_Residuals*Included, axis=0)
    Z_infer_max = np.max(Rescaled_Residuals*NotIncluded, axis=0)
    S_USDR_minmax = Z_infer_max - Z_train_min
    S_USDR_minmax = minmax_scale(S_USDR_minmax, feature_range=(0,1))

    # Compute Blind Ensemble Mean score
    S_ensemble = np.mean(Rescaled_Residuals, axis=0)
    S_ensemble = minmax_scale(S_ensemble, feature_range=(0,1))

    # Compute Blind Ensemble Max score 
    S_ensemble_max = np.max(Rescaled_Residuals, axis=0)
    S_ensemble_max = minmax_scale(S_ensemble_max, feature_range=(0,1))

    return S_USDR, S_USDR_minmax, Z_infer, Z_train, Rescaled_Residuals, std_j, mu_j, Included, NotIncluded, Y_hat, Y, S_ensemble, S_ensemble_max




In [6]:
#### BACKUP
#def train_and_predict_on_training_set_USDR(coreset_sampling_ratio, run, category, cont_ratio):
#
#    seed_everything(run, workers=True)
#    train_dataset = MVTecDataset_contaminated(
#                    task=TaskType.CLASSIFICATION,
#                    split=Split.TRAIN,
#                    category=category,
#                    cont_ratio=cont_ratio,
#                    run=run,
#                    idx = []           
#                )
#
#    # Create indices for k disjoint datasets
#    train_dataset_length = train_dataset.__len__()
#
#    # Define hyperparameters
#    N = train_dataset_length#23
#    M = 8#16
#    M_train = int(M/2)
#
#    # Create subset boolean matrix
#    ones = np.ones(M_train, dtype=bool)
#    zeros = np.zeros(M-M_train, dtype=bool)
#    period_bool = np.hstack((ones, zeros))
#    subset1_bool = np.array([], dtype=bool)
#    for i in range(int(np.ceil(N/M))):
#        subset1_bool = np.hstack((subset1_bool, period_bool))
#    subsets_bool = subset1_bool
#    for m in range(1,M):
#        subset_new_bool = np.roll(subset1_bool, shift=m)
#        subsets_bool = np.vstack((subsets_bool, subset_new_bool))
#    subsets_bool = subsets_bool[:,:int(np.floor(N/M)*M + np.mod(N, M))]
#    print(subsets_bool.astype(int))
#
#    # Initialize arrays for predictions and subsets
#    predictions_subsets_arr = np.empty(M, dtype=object)
#    subsets_bool_sorted_arr = np.empty((M,N))
#
#    # Create indices for M partially overlapping subsets
#    indices = np.arange(0, train_dataset_length)
#    np.random.seed(run)
#    np.random.shuffle(indices)
#
#    # Train M models on M partially overlapping subsets
#    for m_iter in range(M):
#
#        # Define indices for M partially overlapping subsets
#        indices_subset = np.sort(indices[np.where(subsets_bool.astype(int)[m_iter])])
#        
#        # Train model on subset
#        seed_everything(run, workers=True)
#        datamodule = MVTec_contaminated(category=category, cont_ratio=cont_ratio, run=run, idx=indices_subset)
#        model = Patchcore(coreset_sampling_ratio=coreset_sampling_ratio) 
#        engine = Engine(task=TaskType.CLASSIFICATION, image_metrics=["AUROC", "AUPR", "F1Score"], max_epochs=10, devices=1)
#        engine.fit(datamodule=datamodule, model=model)
#
#        # Predict complete training set
#        predictions_subset = engine.predict(model=model, dataset=train_dataset)
#
#        # Save the predictions
#        predictions_subsets_arr[m_iter] = predictions_subset
#
#        # Save the subset
#        subsets_bool_sorted = np.zeros(N)
#        subsets_bool_sorted[indices_subset] = 1
#        subsets_bool_sorted_arr[m_iter,:] = subsets_bool_sorted
#
#    # Extract 'pred_scores' from each list and stack them
#    pred_scores_list = []
#    for sublist in predictions_subsets_arr:
#        stacked_sublist_scores = np.array([d['pred_scores'].item() for d in sublist])
#        pred_scores_list.append(stacked_sublist_scores)
#    pred_scores_arr = np.vstack(pred_scores_list)
#
#    # Extract labels
#    label_arr = np.array([item['label'].item() for item in predictions_subsets_arr[0]])
#
#    # Compute USDR scores
#    Y = label_arr 
#    Y_hat = pred_scores_arr
#    Residuals = abs(Y_hat) # abs(Y - Y_hat)
#    Included = subsets_bool_sorted_arr
#    NotIncluded = 1-subsets_bool_sorted_arr
#    N_j = np.sum(Included, axis=1)
#    mu_j = np.empty([M])
#    std_j = np.empty([M])
#    for j in range(M):
#        mu_j[j] = np.mean(Residuals[j,:][Included[j,:]==1])
#        std_j[j] = np.std(Residuals[j,:][Included[j,:]==1])
#    Rescaled_Residuals = (Residuals - mu_j[:, np.newaxis])/std_j[:, np.newaxis]
#    Z_train = 1/M_train*np.sum(Rescaled_Residuals*Included, axis=0)
#    Z_infer = 1/(M-M_train)*np.sum(Rescaled_Residuals*NotIncluded, axis=0)
#    S_USDR = Z_infer - Z_train
#    S_USDR = minmax_scale(S_USDR, feature_range=(0,1))
#
#    return S_USDR, Z_infer, Z_train, Rescaled_Residuals, std_j, mu_j, Included, NotIncluded, Y_hat, Y
#



In [7]:
# Inputs
coreset_sampling_ratio = 0.01
run = 1
category = "screw"#"wood" #"cable" #"metal_nut" #  
cont_ratio = 0.15


In [8]:
S_blind, Y_blind = train_and_predict_on_training_set_blind(coreset_sampling_ratio, run, category, cont_ratio)

[rank: 0] Seed set to 1
/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/wueesmat/anaconda3/envs/anomalib_env/lib/pytho ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

Training: |                                                                                                   …

/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/lightning/pytorch/loops/optimization/automatic.py:129: `training_step` returned `None`. If this was on purpose, ignore this warning...


Validation: |                                                                                                 …

Output()

`Trainer.fit` stopped: `max_epochs=1` reached.
[rank: 0] Seed set to 1
ckpt_path is not provided. Model weights will not be loaded.
/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/wueesmat/anaconda3/envs/anomalib_env/lib/pytho ...
/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/wueesmat/anaconda3/envs/anomalib_env/

Predicting: |                                                                                                 …

In [9]:
S_clean, Y_clean = train_and_predict_on_training_set_clean(coreset_sampling_ratio, run, category, cont_ratio)

[rank: 0] Seed set to 1
/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/wueesmat/anaconda3/envs/anomalib_env/lib/pytho ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

Training: |                                                                                                   …

Validation: |                                                                                                 …

Output()

`Trainer.fit` stopped: `max_epochs=1` reached.
[rank: 0] Seed set to 1
ckpt_path is not provided. Model weights will not be loaded.
/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/wueesmat/anaconda3/envs/anomalib_env/lib/pytho ...
/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/wueesmat/anaconda3/envs/anomalib_env/

Predicting: |                                                                                                 …

In [10]:
S_USDR, S_USDR_minmax, Z_infer, Z_train, Rescaled_Residuals, std_j, mu_j, Included, NotIncluded, Y_hat, Y, S_ensemble, S_ensemble_max = train_and_predict_on_training_set_USDR(coreset_sampling_ratio, run, category, cont_ratio)

[rank: 0] Seed set to 1
[rank: 0] Seed set to 1


[[1 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 0 1 ... 1 0 0]
 ...
 [1 0 0 ... 1 1 1]
 [1 1 0 ... 0 1 1]
 [1 1 1 ... 0 0 1]]


/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/wueesmat/anaconda3/envs/anomalib_env/lib/pytho ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/wueesmat/anaconda

Training: |                                                                                                   …

Validation: |                                                                                                 …

...

Training: |                                                                                                   …

Validation: |                                                                                                 …

Output()

`Trainer.fit` stopped: `max_epochs=1` reached.
ckpt_path is not provided. Model weights will not be loaded.
/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/wueesmat/anaconda3/envs/anomalib_env/lib/pytho ...
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/wueesmat/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Predicting: |                                                                                                 …

In [ ]:
# S_USDR = np.array([-1 , 1, 2, 3])
# from sklearn.preprocessing import minmax_scale
# 
# S_USDR_scaled = minmax_scale(S_USDR, feature_range=(0,1))
# S_USDR_scaled

In [ ]:
np.max(S_ensemble)

In [ ]:
S_ensemble_old = S_ensemble

In [ ]:
S_ensemble = S_ensemble_old

In [ ]:
S_ensemble = S_ensemble_max #S_ensemble = S_ensemble_max

In [ ]:
S_ensemble ==S_ensemble_max

In [ ]:
S_USDR = S_USDR_old

In [ ]:
S_USDR_old = S_USDR

In [ ]:

S_USDR = S_USDR_minmax

In [ ]:
#S_ensemble = minmax_scale(S_ensemble, feature_range=(0,1))

In [ ]:
# Prepare Average Precision 
from sklearn.metrics import precision_recall_curve, average_precision_score, PrecisionRecallDisplay

# Blind
precision_blind, recall_blind, _ = precision_recall_curve(Y_blind, S_blind)
AP_blind = average_precision_score(Y_blind, S_blind)

# Ensemble Blind
precision_ensemble, recall_ensemble, _ = precision_recall_curve(Y, S_ensemble)
AP_ensemble = average_precision_score(Y, S_ensemble)

# USDR
precision_USDR, recall_USDR, _ = precision_recall_curve(Y, S_USDR)
AP_USDR = average_precision_score(Y, S_USDR)

# Clean
precision_clean, recall_clean, _ = precision_recall_curve(Y_clean, S_clean)
AP_clean = average_precision_score(Y_clean, S_clean)

In [ ]:
from skimage.filters import threshold_otsu, threshold_triangle, threshold_minimum
from sklearn.metrics import precision_score, recall_score

# https://bioimagebook.github.io/chapters/2-processing/3-thresholding/thresholding.html
thresh_method = "minimum"

if thresh_method=="otsu":
    # Compute Otsu threshold
    thresh_blind = threshold_otsu(S_blind)
    thresh_ensemble = threshold_otsu(S_ensemble)
    thresh_USDR = threshold_otsu(S_USDR)
    thresh_clean = threshold_otsu(S_clean)
elif thresh_method=="triangle":
    # Compute triangle threshold
    thresh_blind = threshold_triangle(S_blind)
    thresh_ensemble = threshold_triangle(S_ensemble)
    thresh_USDR = threshold_triangle(S_USDR)
    thresh_clean = threshold_triangle(S_clean)
elif thresh_method=="minimum":
    # Compute minimum threshold
    thresh_blind = threshold_minimum(S_blind)
    thresh_ensemble = threshold_minimum(S_ensemble)
    thresh_USDR = threshold_minimum(S_USDR)
    thresh_clean = threshold_minimum(S_clean)
else:
    print("Method not implemented")

# Estimate anomaly ratio
qratio_blind = np.sum(S_blind>thresh_blind) / len(S_blind)
qratio_ensemble = np.sum(S_ensemble>thresh_ensemble) / len(S_ensemble)
qratio_USDR = np.sum(S_USDR>thresh_USDR) / len(S_USDR)
qratio_clean = np.sum(S_clean>thresh_clean) / len(S_clean)

# # Classify as normal or abnormal based on threshold
# Y_blind_thresh = np.zeros(len(S_blind))
# Y_blind_thresh[S_blind>thresh_blind] = 1
# Y_ensemble_thresh = np.zeros(len(S_ensemble))
# Y_ensemble_thresh[S_ensemble>thresh_ensemble] = 1
# Y_USDR_thresh = np.zeros(len(S_USDR))
# Y_USDR_thresh[S_USDR>thresh_USDR] = 1
# Y_clean_thresh = np.zeros(len(S_clean))
# Y_clean_thresh[S_clean>thresh_clean] = 1

# Classify as normal or abnormal based on threshold
safety_factor = 1
qthresh_blind = np.quantile(S_blind, 1-safety_factor*qratio_blind)
qthresh_ensemble = np.quantile(S_ensemble, 1-safety_factor*qratio_ensemble)
qthresh_USDR = np.quantile(S_USDR, 1-safety_factor*qratio_USDR)
qthresh_clean = np.quantile(S_clean, 1-safety_factor*qratio_clean)
Y_blind_thresh = np.where(S_blind < qthresh_blind, 0, 1)
Y_ensemble_thresh = np.where(S_ensemble < qthresh_ensemble, 0, 1)
Y_USDR_thresh = np.where(S_USDR < qthresh_USDR, 0, 1)
Y_clean_thresh = np.where(S_clean < qthresh_clean, 0, 1)

# Compute precision and recall
precision_blind_thresh = precision_score(Y_blind, Y_blind_thresh)
recall_blind_thresh = recall_score(Y_blind, Y_blind_thresh)
precision_ensemble_thresh = precision_score(Y, Y_ensemble_thresh)
recall_ensemble_thresh = recall_score(Y, Y_ensemble_thresh)
precision_USDR_thresh = precision_score(Y, Y_USDR_thresh)
recall_USDR_thresh = recall_score(Y, Y_USDR_thresh)
precision_clean_thresh = precision_score(Y_clean, Y_clean_thresh)
recall_clean_thresh = recall_score(Y_clean, Y_clean_thresh)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# Plot setttings
fontsize = 8
markersize = 2

fig, axs = plt.subplots(1, 7, figsize=(1.5*14,1.5*2), constrained_layout=True)

# Blind
axs[0].grid()
axs[0].hist(S_blind[Y_blind==0], bins=100, range=(0,1), label="True normal", color='C8')
axs[0].hist(S_blind[Y_blind==1], bins=100, range=(0,1), label="True abnormal", color='C5')
axs[0].axvline(thresh_blind, label="Threshold", color='C7')
axs[0].axvline(qthresh_blind, label="Applied", color='C7')
axs[0].legend(fontsize=fontsize)
axs[0].set_title("Blind", fontsize=fontsize, fontweight="bold")
axs[0].set_ylim([0, 50])
axs[0].set_xlabel("Score [-]", fontsize=fontsize)
axs[0].set_ylabel("Count [-]", fontsize=fontsize)
axs[0].xaxis.set_tick_params(labelsize=fontsize)
axs[0].yaxis.set_tick_params(labelsize=fontsize)

# Blind Ensemble 
axs[1].grid()
axs[1].hist(S_ensemble[Y==0], bins=100, range=(0,1), label="True normal", color='C8')
axs[1].hist(S_ensemble[Y==1], bins=100, range=(0,1), label="True abnormal", color='C5')
axs[1].axvline(thresh_ensemble, label="Threshold", color='C7', linestyle='--')
axs[1].axvline(qthresh_ensemble, label="Applied", color='C7')
axs[1].legend(fontsize=fontsize)
axs[1].set_title("Ensemble", fontsize=fontsize, fontweight="bold")
axs[1].set_ylim([0, 50])
axs[1].set_xlabel("Score [-]", fontsize=fontsize)
axs[1].set_ylabel("Count [-]", fontsize=fontsize)
axs[1].xaxis.set_tick_params(labelsize=fontsize)
axs[1].yaxis.set_tick_params(labelsize=fontsize)

# USDR
axs[2].grid()
axs[2].hist(S_USDR[Y==0], bins=100, range=(0,1), label="True normal", color='C8')
axs[2].hist(S_USDR[Y==1], bins=100, range=(0,1), label="True abnormal", color='C5')
axs[2].axvline(thresh_USDR, label="Threshold", color='C7', linestyle='--')
axs[2].axvline(qthresh_USDR, label="Applied", color='C7')
axs[2].legend(fontsize=fontsize)
axs[2].set_title("USDR", fontsize=fontsize, fontweight="bold")
axs[2].set_ylim([0, 50])
axs[2].set_xlabel("Score [-]", fontsize=fontsize)
axs[2].set_ylabel("Count [-]", fontsize=fontsize)
axs[2].xaxis.set_tick_params(labelsize=fontsize)
axs[2].yaxis.set_tick_params(labelsize=fontsize)

# Clean
axs[3].grid()
axs[3].hist(S_clean[Y_clean==0], bins=100, range=(0,1), label="True normal", color='C8')
axs[3].hist(S_clean[Y_clean==1], bins=100, range=(0,1), label="True abnormal", color='C5')
axs[3].axvline(thresh_clean, label="Threshold", color='C7', linestyle='--')
axs[3].axvline(qthresh_clean, label="Applied", color='C7')
axs[3].legend(fontsize=fontsize)
axs[3].set_title("Clean", fontsize=fontsize, fontweight="bold")
axs[3].set_ylim([0, 50])
axs[3].set_xlabel("Score [-]", fontsize=fontsize)
axs[3].set_ylabel("Count [-]", fontsize=fontsize)
axs[3].xaxis.set_tick_params(labelsize=fontsize)
axs[3].yaxis.set_tick_params(labelsize=fontsize)

# Estimated anomaly ratio
axs[4].bar(["Blind", "Ensemble", "USDR", "Clean"], [qratio_blind, qratio_ensemble, qratio_USDR, qratio_clean], color=['C0', 'C2', 'C3', 'C1'])#, color=['C7', 'C7', 'C7', 'C7'])
axs[4].axhline(cont_ratio, label="True anomaly ratio", linestyle='--', color='black')
axs[4].legend(fontsize=fontsize)
axs[4].xaxis.set_tick_params(labelsize=fontsize)
axs[4].yaxis.set_tick_params(labelsize=fontsize)
axs[4].set_ylabel("Estimated anomaly ratio [-]", fontsize=fontsize)
axs[4].set_ylim([-0.05, 1.05])
#axs[4].set_ylim([-0.025, 0.50])

# Precision Recall Curves
#disp = PrecisionRecallDisplay(precision=precision_blind, recall=recall_blind)
axs[5].plot(recall_blind, precision_blind, label="$S^{blind}$", color='C0')
axs[5].plot(recall_blind_thresh, precision_blind_thresh, color='C0', marker='D', markersize=3*markersize)
axs[5].plot(recall_ensemble, precision_ensemble, label="$S^{ensemble}$", color='C2')
axs[5].plot(recall_ensemble_thresh, precision_ensemble_thresh, color='C2', marker='D', markersize=3*markersize)
axs[5].plot(recall_USDR, precision_USDR, label="$S^{USDR}$", color='C3')
axs[5].plot(recall_USDR_thresh, precision_USDR_thresh, color='C3', marker='D', markersize=3*markersize)
axs[5].plot(recall_clean, precision_clean, label="$S^{clean}$", color='C1')
axs[5].plot(recall_clean_thresh, precision_clean_thresh, color='C1', marker='D', markersize=3*markersize)
axs[5].legend(fontsize=fontsize)
axs[5].xaxis.set_tick_params(labelsize=fontsize)
axs[5].yaxis.set_tick_params(labelsize=fontsize)
axs[5].set_xlabel("Recall [-]", fontsize=fontsize)
axs[5].set_ylabel("Precision [-]", fontsize=fontsize)
axs[5].set_xlim([-0.05, 1.05])
axs[5].set_ylim([-0.05, 1.05])


# Average Precision Scores Barplot
x = ["Blind", "Ensemble", "USDR", "Clean"]
y = [AP_blind, AP_ensemble, AP_USDR, AP_clean]
axs[6].bar(x, y, color=['C0', 'C2', 'C3', 'C1'])
for i, v in enumerate(y):
    axs[6].text(i, v + 0.01, str(round(v,3)), ha='center', fontsize=fontsize)

axs[6].xaxis.set_tick_params(labelsize=fontsize)
axs[6].yaxis.set_tick_params(labelsize=fontsize)
axs[6].set_ylabel("Avg. Precision [-]", fontsize=fontsize)
axs[6].set_ylim([-0.05, 1.05])



In [ ]:
AP_USDR

In [ ]:
AP_ensemble

In [ ]:
AP_ensemble





In [ ]:

## Blind
#axs[0].plot(S_blind, label="$S^{blind}$", color='grey', linestyle='', marker="o", markersize=markersize)
#axs[0].plot(np.where(Y_blind==0)[0], Y_blind[Y_blind==0], label="True normal", color='C2', linestyle='', marker="o", markersize=markersize)
#axs[0].plot(np.where(Y_blind==1)[0], Y_blind[Y_blind==1], label="True abnormal", color='C3', linestyle='', marker="o", markersize=markersize)
#axs[0].legend(fontsize=fontsize)
#axs[0].grid()
#axs[0].xaxis.set_tick_params(labelsize=fontsize)
#axs[0].yaxis.set_tick_params(labelsize=fontsize)
#axs[0].set_xlabel("Index [-]", fontsize=fontsize)
#axs[0].set_ylabel("Score [-]", fontsize=fontsize)
#
#
## Blind Ensemble 
#axs[1].plot(S_ensemble, label="$S^{ensemble}$", color='grey', linestyle='', marker="o", markersize=markersize)
#axs[1].plot(np.where(Y==0)[0], Y[Y==0], label="True normal", color='C2', linestyle='', marker="o", markersize=markersize)
#axs[1].plot(np.where(Y==1)[0], Y[Y==1], label="True abnormal", color='C3', linestyle='', marker="o", markersize=markersize)
#axs[1].legend(fontsize=fontsize)
#axs[1].grid()
#axs[1].xaxis.set_tick_params(labelsize=fontsize)
#axs[1].yaxis.set_tick_params(labelsize=fontsize)
#axs[1].set_xlabel("Index [-]", fontsize=fontsize)
#axs[1].set_ylabel("Score [-]", fontsize=fontsize)
#
## USDR
#axs[2].plot(S_USDR, label="$S^{USDR}$", color='grey', linestyle='', marker="o", markersize=markersize)
#axs[2].plot(np.where(Y==0)[0], Y[Y==0], label="True normal", color='C2', linestyle='', marker="o", markersize=markersize)
#axs[2].plot(np.where(Y==1)[0], Y[Y==1], label="True abnormal", color='C3', linestyle='', marker="o", markersize=markersize)
#axs[2].legend(fontsize=fontsize)
#axs[2].grid()
#axs[2].xaxis.set_tick_params(labelsize=fontsize)
#axs[2].yaxis.set_tick_params(labelsize=fontsize)
#axs[2].set_xlabel("Index [-]", fontsize=fontsize)
#axs[2].set_ylabel("Score [-]", fontsize=fontsize)
#
#
## Clean
#axs[3].plot(S_clean, label="$S^{clean}$", color='grey', linestyle='', marker="o", markersize=markersize)
#axs[3].plot(np.where(Y_clean==0)[0], Y_clean[Y_clean==0], label="True normal", color='C2', linestyle='', marker="o", markersize=markersize)
#axs[3].plot(np.where(Y_clean==1)[0], Y_clean[Y_clean==1], label="True abnormal", color='C3', linestyle='', marker="o", markersize=markersize)
#axs[3].legend(fontsize=fontsize)
#axs[3].grid()
#axs[3].xaxis.set_tick_params(labelsize=fontsize)
#axs[3].yaxis.set_tick_params(labelsize=fontsize)
#axs[3].set_xlabel("Index [-]", fontsize=fontsize)
#axs[3].set_ylabel("Score [-]", fontsize=fontsize)

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(10, 2.5), constrained_layout=True)

# Blind
axs[0].grid()
axs[0].hist(S_blind[Y_blind==0], bins=100, range=(0,1), label="True normal", color='C8')
axs[0].hist(S_blind[Y_blind==1], bins=100, range=(0,1), label="True abnormal", color='C5')
axs[0].axvline(thresh_blind, label="Otsu", color='C7')
axs[0].legend(fontsize=fontsize)
axs[0].set_ylim([0, 50])
axs[0].set_xlabel("Score [-]", fontsize=fontsize)
axs[0].set_ylabel("Count [-]", fontsize=fontsize)
axs[0].set_title("Blind (" + str(round(qratio_blind,3)) + ")", fontsize=fontsize, fontweight="bold")
axs[0].xaxis.set_tick_params(labelsize=fontsize)
axs[0].yaxis.set_tick_params(labelsize=fontsize)

# Blind Ensemble 
axs[1].grid()
axs[1].hist(S_ensemble[Y==0], bins=100, range=(0,1), label="True normal", color='C8')
axs[1].hist(S_ensemble[Y==1], bins=100, range=(0,1), label="True abnormal", color='C5')
axs[1].axvline(threshold_otsu(S_ensemble), label="Otsu", color='C7')
axs[1].legend(fontsize=fontsize)
axs[1].set_ylim([0, 50])
axs[1].set_xlabel("Score [-]", fontsize=fontsize)
axs[1].set_ylabel("Count [-]", fontsize=fontsize)
axs[1].set_title("Ensemble (" + str(round(qratio_ensemble,3)) + ")", fontsize=fontsize, fontweight="bold")
axs[1].xaxis.set_tick_params(labelsize=fontsize)
axs[1].yaxis.set_tick_params(labelsize=fontsize)

# USDR
axs[2].grid()
axs[2].hist(S_USDR[Y==0], bins=100, range=(0,1), label="True normal", color='C8')
axs[2].hist(S_USDR[Y==1], bins=100, range=(0,1), label="True abnormal", color='C5')
axs[2].axvline(thresh_USDR, label="Otsu", color='C7')
axs[2].legend(fontsize=fontsize)
axs[2].set_ylim([0, 50])
axs[2].set_xlabel("Score [-]", fontsize=fontsize)
axs[2].set_ylabel("Count [-]", fontsize=fontsize)
axs[2].set_title("USDR (" + str(round(qratio_USDR,3)) + ")", fontsize=fontsize, fontweight="bold")
axs[2].xaxis.set_tick_params(labelsize=fontsize)
axs[2].yaxis.set_tick_params(labelsize=fontsize)

# Clean
axs[3].grid()
axs[3].hist(S_clean[Y_clean==0], bins=100, range=(0,1), label="True normal", color='C8')
axs[3].hist(S_clean[Y_clean==1], bins=100, range=(0,1), label="True abnormal", color='C5')
axs[3].axvline(thresh_clean, label="Otsu", color='C7')
axs[3].legend(fontsize=fontsize)
axs[3].set_ylim([0, 50])
axs[3].set_xlabel("Score [-]", fontsize=fontsize)
axs[3].set_ylabel("Count [-]", fontsize=fontsize)
axs[3].set_title("Clean (" + str(round(qratio_clean,3)) + ")", fontsize=fontsize, fontweight="bold")
axs[3].xaxis.set_tick_params(labelsize=fontsize)
axs[3].yaxis.set_tick_params(labelsize=fontsize)

plt.show()



In [ ]:
## Compute USDR scores
#
#Y = label_arr 
#Y_hat = pred_scores_arr
#Residuals = abs(Y_hat) # abs(Y - Y_hat)
#Included = subsets_bool_sorted_arr
#NotIncluded = 1-subsets_bool_sorted_arr
#N_j = np.sum(Included, axis=1)
#
#mu_j = np.empty([M])
#std_j = np.empty([M])
#for j in range(M):
#    mu_j[j] = np.mean(Residuals[j,:][Included[j,:]==1])
#    std_j[j] = np.std(Residuals[j,:][Included[j,:]==1])
#
#Rescaled_Residuals = (Residuals - mu_j[:, np.newaxis])/std_j[:, np.newaxis]
#Z_train = 1/M_train*np.sum(Rescaled_Residuals*Included, axis=0)
#Z_infer = 1/(M-M_train)*np.sum(Rescaled_Residuals*NotIncluded, axis=0)
#S_USDR = Z_infer - Z_train
#
#print(Y.shape)
#print(Y_hat.shape)
#print(Residuals.shape)
#print(Included.shape)
#print(N_j.shape)
#print(mu_j.shape)
#print(std_j.shape)
#print(Rescaled_Residuals.shape)
#print(Z_train.shape)
#print(Z_infer.shape)
#print(S_USDR.shape)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# Plot setttings
fontsize = 8
markersize = 2

M, N = Y_hat.shape

fig, axs = plt.subplots(M, 4, figsize=(12,2*M), constrained_layout=True)
for m_iter in range(M):


    # Anomaly scores
    axs[m_iter,0].plot(np.where(NotIncluded[m_iter,:]==1)[0], Y_hat[m_iter,:][NotIncluded[m_iter,:]==1], label="Not in subset", color="grey", linestyle='', marker="o", markersize=markersize)
    axs[m_iter,0].plot(np.where(Included[m_iter,:]==1)[0], Y_hat[m_iter,:][Included[m_iter,:]==1], label="In subset", color="C9", linestyle='', marker="o", markersize=markersize)
    axs[m_iter,0].plot(np.where(Y==0)[0], Y[Y==0], label="True normal", color='C2', linestyle='', marker="o", markersize=markersize)
    axs[m_iter,0].plot(np.where(Y==1)[0], Y[Y==1], label="True abnormal", color='C3', linestyle='', marker="o", markersize=markersize)
    axs[m_iter,0].hlines(mu_j[m_iter], 0, N, color="C9", linestyle='-')
    axs[m_iter,0].fill_between([0, N], mu_j[m_iter] - 2*std_j[m_iter], mu_j[m_iter] + 2*std_j[m_iter], color="C9", alpha=0.3, linewidth=0.0)
    axs[m_iter,0].set_ylim([-0.1, 1.1])
    axs[m_iter,0].grid()
    axs[m_iter,0].xaxis.set_tick_params(labelsize=fontsize)
    axs[m_iter,0].yaxis.set_tick_params(labelsize=fontsize)
    axs[m_iter,0].set_xlabel("Index [-]", fontsize=fontsize)
    axs[m_iter,0].set_ylabel("Score [-]", fontsize=fontsize)


    # Rescaled residuals
    axs[m_iter,1].plot(np.where(NotIncluded[m_iter,:]==1)[0], Rescaled_Residuals[m_iter,:][NotIncluded[m_iter,:]==1], label="Not in subset", color='grey', linestyle='', marker="o", markersize=markersize)
    axs[m_iter,1].plot(np.where(Included[m_iter,:]==1)[0], Rescaled_Residuals[m_iter,:][Included[m_iter,:]==1], label="In subset", color='C9', linestyle='', marker="o", markersize=markersize)
    axs[m_iter,1].plot(np.where(Y==0)[0], Y[Y==0]-5, label="True normal", color='C2', linestyle='', marker="o", markersize=markersize)
    axs[m_iter,1].plot(np.where(Y==1)[0], 35*Y[Y==1], label="True abnormal", color='C3', linestyle='', marker="o", markersize=markersize)
    axs[m_iter,1].set_ylim([-10, 40])
    axs[m_iter,1].grid()
    axs[m_iter,1].xaxis.set_tick_params(labelsize=fontsize)
    axs[m_iter,1].yaxis.set_tick_params(labelsize=fontsize)
    axs[m_iter,1].set_xlabel("Index [-]", fontsize=fontsize)
    axs[m_iter,1].set_ylabel("Score [-]", fontsize=fontsize)

    if m_iter==0:
        axs[m_iter,0].legend(fontsize=fontsize)

        axs[m_iter,1].legend(fontsize=fontsize)

        axs[m_iter,2].plot(Z_infer, label="$z^{infer}$", color='grey', linestyle='', marker="o", markersize=markersize)
        axs[m_iter,2].plot(Z_train, label="$z^{train}$", color='C9', linestyle='', marker="o", markersize=markersize)
        axs[m_iter,2].plot(np.where(Y==0)[0], Y[Y==0]-5, label="True normal", color='C2', linestyle='', marker="o", markersize=markersize)
        axs[m_iter,2].plot(np.where(Y==1)[0], 35*Y[Y==1], label="True abnormal", color='C3', linestyle='', marker="o", markersize=markersize)
        axs[m_iter,2].set_ylim([-10, 40])
        axs[m_iter,2].legend(fontsize=fontsize)
        axs[m_iter,2].grid()
        axs[m_iter,2].xaxis.set_tick_params(labelsize=fontsize)
        axs[m_iter,2].yaxis.set_tick_params(labelsize=fontsize)
        axs[m_iter,2].set_xlabel("Index [-]", fontsize=fontsize)
        axs[m_iter,2].set_ylabel("Score [-]", fontsize=fontsize)


        # USDR scores
        axs[m_iter,3].plot(S_USDR, label="$S^{USDR}$", color='grey', linestyle='', marker="o", markersize=markersize)
        axs[m_iter,3].plot(np.where(Y==0)[0], Y[Y==0], label="True normal", color='C2', linestyle='', marker="o", markersize=markersize)
        axs[m_iter,3].plot(np.where(Y==1)[0], Y[Y==1], label="True abnormal", color='C3', linestyle='', marker="o", markersize=markersize)
        axs[m_iter,3].set_ylim([-0.05, 1.05])
        axs[m_iter,3].legend(fontsize=fontsize)
        axs[m_iter,3].grid()
        axs[m_iter,3].xaxis.set_tick_params(labelsize=fontsize)
        axs[m_iter,3].yaxis.set_tick_params(labelsize=fontsize)
        axs[m_iter,3].set_xlabel("Index [-]", fontsize=fontsize)
        axs[m_iter,3].set_ylabel("Score [-]", fontsize=fontsize)

        # Set titles
        axs[m_iter,0].set_title("Anomaly scores", fontsize=fontsize, fontweight="bold")
        axs[m_iter,1].set_title("Rescaled anomaly scores", fontsize=fontsize, fontweight="bold")
        axs[m_iter,2].set_title("Mean scaled anomaly scores", fontsize=fontsize, fontweight="bold")
        axs[m_iter,3].set_title("USDR scores", fontsize=fontsize, fontweight="bold")
        
    elif m_iter==1:
        axs[m_iter,3].plot(S_ensemble, label="$S^{ensemble}$", color='grey', linestyle='', marker="o", markersize=markersize)
        axs[m_iter,3].plot(np.where(Y==0)[0], Y[Y==0], label="True normal", color='C2', linestyle='', marker="o", markersize=markersize)
        axs[m_iter,3].plot(np.where(Y==1)[0], Y[Y==1], label="True abnormal", color='C3', linestyle='', marker="o", markersize=markersize)
        axs[m_iter,3].set_ylim([-0.05, 1.05])
        axs[m_iter,3].legend(fontsize=fontsize)
        axs[m_iter,3].grid()
        axs[m_iter,3].xaxis.set_tick_params(labelsize=fontsize)
        axs[m_iter,3].yaxis.set_tick_params(labelsize=fontsize)
        axs[m_iter,3].set_xlabel("Index [-]", fontsize=fontsize)
        axs[m_iter,3].set_ylabel("Score [-]", fontsize=fontsize)
        axs[m_iter,3].set_title("Ensemble scores", fontsize=fontsize, fontweight="bold")
        axs[1,2].axis('off')


    elif m_iter>0:
        # Mean residuals
        axs[m_iter,2].axis('off')
        axs[m_iter,3].axis('off')





In [ ]:
S_USDR

In [ ]:
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay, average_precision_score

y_true = Y
y_scores = S_USDR
precision, recall, thresholds = precision_recall_curve(
    y_true, y_scores)

m_iter = 0
y_true = Y
y_scores = S_USDR
precision, recall, thresholds = precision_recall_curve(
    y_true, y_scores)
average_precision = average_precision_score(y_true, y_scores)
disp = PrecisionRecallDisplay(precision=precision, recall=recall, average_precision=average_precision)
disp.plot()
plt.show()




In [ ]:
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay, average_precision_score
### Precision Recall Curve

m_iter = 1
y_true = Y[NotIncluded[m_iter,:]==1]
y_scores = Y_hat[m_iter,:][NotIncluded[m_iter,:]==1]
#y_scores[y_scores > 0.49] = 1
#y_scores[y_scores <= 0.49] = 0
precision, recall, thresholds = precision_recall_curve(
    y_true, y_scores)
average_precision = average_precision_score(y_true, y_scores)
disp = PrecisionRecallDisplay(precision=precision, recall=recall, average_precision=average_precision)
disp.plot()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# Plot setttings
fontsize = 8
markersize = 2

# Prepare plot
%matplotlib inline
fig, axs = plt.subplots(2, 1, figsize=(8,6), constrained_layout=True)

for m_iter in  range(M):

    # Prepare data
    labels = np.array([item['label'].item() for item in predictions_subsets_arr[m_iter]])
    labels0x = np.where(labels==0)[0]
    labels0y = labels[labels==0]
    labels1x = np.where(labels==1)[0]
    labels1y = labels[labels==1]
    
    subsets_bool_sorted = subsets_bool_sorted_arr[m_iter]
    pred_scores = np.array([item['pred_scores'][0] for item in predictions_subsets_arr[m_iter]])
    
    pred_scores_in_x = np.where(subsets_bool_sorted.astype(bool))[0]
    pred_scores_in_y = pred_scores[subsets_bool_sorted.astype(bool)]
    pred_scores_out_x = np.where(~subsets_bool_sorted.astype(bool))[0]
    pred_scores_out_y = pred_scores[~subsets_bool_sorted.astype(bool)]

    # Plot
    m = 0#m_iter
    axs[m].plot(labels0x, labels0y, label="True normal", color='C2', linestyle='', marker="o", markersize=markersize)
    axs[m].plot(labels1x, labels1y, label="True abnormal", color='C3', linestyle='', marker="o", markersize=markersize)
    axs[m].plot(pred_scores_in_x, pred_scores_in_y, label="Prediction Scores (in)", color='C0', linestyle='', marker="o", markersize=markersize)
    axs[m].plot(pred_scores_out_x, pred_scores_out_y, label="Prediction Scores (out)", color='C1', linestyle='', marker="o", markersize=markersize)

    #axs[m_iter].plot(pred_scores, label="Prediction Scores", color='C0', linestyle='', marker="o", markersize=markersize)
    #axs[m].legend(fontsize=fontsize)
axs[m].grid()
axs[m].set_title("Anomaly scores", fontsize=fontsize, fontweight="bold")
axs[m].set_xlim([0, N])
axs[m].set_xlabel("Index [-]", fontsize=fontsize)
axs[m].set_ylabel("Score [-]", fontsize=fontsize)
axs[m].xaxis.set_tick_params(labelsize=fontsize)
axs[m].yaxis.set_tick_params(labelsize=fontsize)
